In [1]:
# Подключение библиотек
from fake_useragent import UserAgent
from bs4 import BeautifulSoup as bs
import requests
import re
import time
import pandas as pd
import random
from datetime import datetime
from urllib.parse import urljoin

In [2]:
# Короткая задержка между последовательными запросами
def get_short_delay():
    return random.uniform(2, 5)

# Средняя задержка после важных действий
def get_medium_delay():
    return random.uniform(7, 15)

# Длинная задержка при подозрении на блокировку
def get_long_delay():
    return random.uniform(30, 90)

# Очень длинная задержка при обнаружении блокировки
def get_extended_delay():
    return random.uniform(120, 300)

### Парсинг рецензий.

In [10]:
# Структуры для хранения данных
reviews_list = {
    'id': [],
    'author_name': [],
    'book_slug': [],
    'title_book': [],
    'review_url': [],
    'author_book': [],
    'date_creation': [],
    'count_view': [],
    'evaluation': [],
    'title_review': [],
    'text_review': [],
    'count_like': [],
    'count_comment': []
}

comments_reviews_list = {
    'id_review': [],
    'id_comment': [],
    'author_comment': [],
    'date_comment': [],
    'text_comment': [],
    'parent_comment': []
}

In [11]:
# Настройки
headers = {'User-Agent': UserAgent().random}

In [12]:
# Парсинг комментариев к рецензии
def parse_review_comments(review_id, review_url):
    try:
        comments_url = f"{review_url}#comments"
        print(f'[{datetime.now().strftime('%H:%M:%S')}] Загрузка комментариев для рецензии {review_id}.')
        
        # Первая задержка перед загрузкой комментариев
        time.sleep(get_medium_delay())

        page = requests.get(comments_url, headers=headers, timeout=20)
        
        if page.status_code != 200:
            print(f'Ошибка загрузки: {page.status_code}.')
            return
        
        soup = bs(page.text, 'html.parser')
        comments = soup.find_all('div', class_=lambda x: x and 'comment-row' in x)
        
        if not comments:
            print('Комментарии не найдены.')
            return
        
        print(f'Найдено {len(comments)} комментариев.')
        
        for i, comment in enumerate(comments, 1):
            try:
                # Задержка между комментариями
                if i % 3 == 0:
                    time.sleep(get_medium_delay())
                else:
                    time.sleep(get_short_delay())
                
                # ID комментария
                comment_id = comment.get('id', '').replace('comment', '') or None
                
                # Уровень вложенности
                comment_level = 1
                level_class = [c for c in comment.get('class', []) if 'comment-level' in c]
                if level_class:
                    level_match = re.search(r'comment-level(\d+)', level_class[0])
                    comment_level = int(level_match.group(1)) if level_match else 1
                
                # Автор
                author_tag = comment.find('a', class_='comment-user-avatar')
                author = author_tag.get('title', 'Аноним') if author_tag else "Аноним"
                
                # Дата
                date_span = comment.find('span', class_='event-user-date')
                date = date_span.get_text(strip=True) if date_span else None
                
                # Текст комментария
                text_div = comment.find('div', class_='commenttext')
                text = None
                if text_div:
                    for img in text_div.find_all('img'):
                        img.decompose()
                    text = ' '.join(text_div.stripped_strings)
                
                # Родительский комментарий
                parent_comment = None
                if comment_level > 1:
                    up_link = comment.find('a', string='Уровень вверх')
                    if up_link:
                        parent_match = re.search(r'comment(\d+)', up_link.get('href', ''))
                        parent_comment = parent_match.group(1) if parent_match else None
                
                # Сохранение данных
                comments_reviews_list['id_review'].append(review_id)
                comments_reviews_list['id_comment'].append(comment_id)
                comments_reviews_list['author_comment'].append(author)
                comments_reviews_list['date_comment'].append(date)
                comments_reviews_list['text_comment'].append(text)
                comments_reviews_list['parent_comment'].append(parent_comment)
            
            except Exception as e:
                print(f'Ошибка обработки комментария: {str(e)} .')
                time.sleep(get_medium_delay())
    
    except Exception as e:
        print(f'Ошибка загрузки комментариев: {str(e)} .')
        time.sleep(get_long_delay())

In [13]:
# Парсинг текста рецензии
def parse_review_text(text_block):
    if not text_block:
        return ''
    
    review_body = text_block.find('div', itemprop='reviewBody')
    if not review_body:
        return ''
    
    # Удаление всех изображений и их обертки
    for img in review_body.find_all('img'):
        img.decompose()
    for a in review_body.find_all('a'):
        if a.find('img'):
            a.decompose()
    
    # Обработка каждого элемента содержимого
    processed_text = []
    
    for element in review_body.children:
        if element.name == 'p':
            # Обработка содержимого параграфа
            para_content = []
            for content in element.contents:
                if content.name in ['b', 'i', 'u', 'a', 'span']:
                    # Сохранение тегов форматирования
                    tag = content.name
                    text = content.get_text()
                    if tag == 'a' or tag == 'span':
                        para_content.append(text)
                    else:
                        para_content.append(f'<{tag}>{text}</{tag}>')
                elif content.name is None:
                    para_content.append(str(content))
            
            # Сбор текста параграфа и добавление <br>
            para_text = ''.join(para_content).strip()
            if para_text:
                processed_text.append(para_text)
                processed_text.append('<br><br>')  # Двойной <br> для абзаца
        
        elif element.name == 'br':
            processed_text.append('<br>')
    
    # Объединение все части
    final_text = ''.join(processed_text)
    
    # Очистка лишние переносы
    final_text = re.sub(r'(<br>\s*){3,}', '<br><br>', final_text)
    final_text = final_text.strip()
    
    return final_text

In [ ]:
# Основная функция парсинга рецензий
def scrape_reviews(start_page=1, max_pages=5):
    pagenum = start_page
    request_count = 0

    for _ in range(max_pages):
        try:
            url = f'https://www.livelib.ru/reviews/~{pagenum}'
            print(f'\n[{datetime.now().strftime('%H:%M:%S')}] Страница {pagenum}: {url} .')

            # Управление задержками
            request_count += 1
            if request_count % 3 == 0:
                delay = get_long_delay()
                print(f'Большая пауза на {delay:.1f} сек...')
                time.sleep(delay)
            else:
                time.sleep(get_medium_delay())

            # Загрузка страницы
            page = requests.get(url, headers=headers, timeout=30)
            print(page.text)

            # Проверка на блокировку
            if 'идёт слишком много запросов' in page.text:
                delay = get_extended_delay()
                print(f'Блокировка. Большая пауза ({delay:.1f} сек).')
                time.sleep(delay)
                continue

            if page.status_code != 200:
                print(f'Ошибка загрузки страницы: {page.status_code}.')
                time.sleep(get_extended_delay())
                continue

            soup = bs(page.text, 'html.parser')
            print(soup)

            # Проверка на пустую страницу
            review_cards = soup.find_all('article', class_=lambda x: x and 'review-card' in x)
            if not review_cards:
                print('Рецензии не найдены, возможно блокировка или последняя страница.')
                time.sleep(get_extended_delay())
                break

            print(f'Найдено {len(review_cards)} рецензий.')

            # Обработка каждой рецензии
            for i, review_card in enumerate(review_cards, 1):
                try:
                    if i % 5 == 0:
                        time.sleep(get_medium_delay())
                    else:
                        time.sleep(get_short_delay())

                    # Базовые данные
                    title_block = review_card.find('h3', class_='lenta-card__title')
                    if not title_block:
                        continue

                    review_link = title_block.find('a', href=True)
                    if not review_link:
                        continue

                    href = review_link['href']
                    match = re.search(r'/review/(\d+)', href)
                    if not match:
                        continue

                    review_id = match.group(1)
                    review_url = urljoin('https://www.livelib.ru', href)

                    # Автор рецензии
                    author_item = review_card.find('a', class_='header-card-user__name')
                    author_name = author_item.get_text(strip=True) if author_item else 'Неизвестно'

                    # Информация о книге
                    book_title_link = review_card.find('a', class_='lenta-card__book-title')
                    title = book_title_link.get_text(strip=True).strip('"') if book_title_link else None

                    book_slug = None
                    if book_title_link:
                        href = book_title_link.get('href', '')
                        book_slug_match = re.search(r'/(?:work|book)/(\d+[-_\w]*)', href)
                        book_slug = book_slug_match.group(1) if book_slug_match else None

                    # Авторы книги
                    author_container = review_card.find('p', class_='lenta-card__author-wrap')
                    author_book = 'Автор неизвестен'
                    if author_container:
                        author_links = author_container.find_all('a', class_='lenta-card__author')
                        authors = [link.get_text(strip=True) for link in author_links if link.get_text(strip=True)]
                        author_book = ', '.join(authors) if authors else author_book

                    # Загрузка страницы рецензии с задержкой
                    time.sleep(get_short_delay())
                    review_page = requests.get(review_url, headers=headers, timeout=30)

                    if review_page.status_code != 200:
                        print(f'Ошибка загрузки рецензии {review_id}: {review_page.status_code}.')
                        time.sleep(get_long_delay())
                        continue

                    review_soup = bs(review_page.text, 'html.parser')

                    # Дата и просмотры
                    review_details = review_soup.find('div', class_='lenta-card__details')
                    date_creation = 'Дата не указана'
                    count_view = '0'

                    if review_details:
                        date_time = review_details.find('p', class_='lenta-card__date')
                        date_creation = date_time.get_text(strip=True).replace('&nbsp;', ' ') if date_time else date_creation

                        count_view_item = review_details.find('p', class_='lenta-card__aliases')
                        count_view = count_view_item.find('span').get_text(strip=True) if count_view_item else count_view
                        
                    # Оценка и заголовок
                    review_title_block = review_soup.find('h1', class_='lenta-card__title')
                    evaluation = '0'
                    title_review = 'Нет названия'

                    if review_title_block:
                        rating_span = review_title_block.find('span', class_='lenta-card__mymark')
                        evaluation = rating_span.get_text(strip=True) if rating_span else evaluation

                        if rating_span:
                            rating_span.decompose()
                        title_review = review_title_block.get_text(strip=True)

                    if not evaluation:
                        continue

                    # Текст рецензии
                    text_block = review_soup.find('div', class_='lenta-card__text')
                    final_text = parse_review_text(text_block) if text_block else ''

                    # Лайки и комментарии
                    footer_container = review_soup.find('div', class_='footer-card__soc-active')
                    like_count = '0'
                    comment_count = '0'

                    if footer_container:
                        like_element = footer_container.find('a', class_='icon-like')
                        if like_element:
                            like_count_span = like_element.find('span', class_='sub__link-count')
                            like_count = like_count_span.get_text(strip=True) if like_count_span else like_count

                        comment_element = footer_container.find('a', class_='icon-comment')
                        if comment_element:
                            comment_count_span = comment_element.find('span')
                            comment_count = comment_count_span.get_text(strip=True) if comment_count_span else comment_count

                    # Парсинг комментариев (если есть)
                    if comment_count != '0':
                        parse_review_comments(review_id, review_url)

                    # Сохранение данных
                    reviews_list['id'].append(review_id)
                    reviews_list['author_name'].append(author_name)
                    reviews_list['book_slug'].append(book_slug)
                    reviews_list['title_book'].append(title)
                    reviews_list['review_url'].append(review_url)
                    reviews_list['author_book'].append(author_book)
                    reviews_list['date_creation'].append(date_creation)
                    reviews_list['count_view'].append(count_view)
                    reviews_list['evaluation'].append(evaluation)
                    reviews_list['title_review'].append(title_review)
                    reviews_list['text_review'].append(final_text)
                    reviews_list['count_like'].append(like_count)
                    reviews_list['count_comment'].append(comment_count)

                    print(f'Обработана рецензия: {review_id}.')

                except Exception as e:
                    print(f'Ошибка обработки рецензии: {str(e)} .')

            pagenum += 1
        
        except Exception as e:
            print(f'Критическая ошибка: {str(e)} .')
            time.sleep(get_extended_delay())
            continue

In [ ]:
print(f'Начало парсинга: {datetime.now().strftime('%H:%M:%S')}')
scrape_reviews(start_page=1, max_pages=5)
print(f'Завершение парсинга: {datetime.now().strftime('%H:%M:%S')}')

Начало парсинга: 16:50:03

[16:50:03] Страница 1: https://www.livelib.ru/reviews/~1 .
Блокировка. Очень большая пауза...

[16:56:05] Страница 1: https://www.livelib.ru/reviews/~1 .
Блокировка. Очень большая пауза...

[17:01:47] Страница 1: https://www.livelib.ru/reviews/~1 .
Большая пауза на 43.5 сек...
Блокировка. Очень большая пауза...

[17:08:57] Страница 1: https://www.livelib.ru/reviews/~1 .
Блокировка. Очень большая пауза...


In [9]:
# Просмотр количества пропусков
print('Количество нулевых значений в reviews_list:')
for i in reviews_list:
    print( i + " - " + str(reviews_list[i].count(None)))

print()

print('Количество нулевых значений в comments_reviews_list:')
for i in comments_reviews_list:
    print( i + " - " + str(comments_reviews_list[i].count(None)))

Количество нулевых значений в reviews_list:
id - 0
author_name - 0
book_slug - 0
title_book - 0
review_url - 0
author_book - 0
date_creation - 0
count_view - 0
evaluation - 0
title_review - 0
text_review - 0
count_like - 0
count_comment - 0

Количество нулевых значений в comments_reviews_list:
id_review - 0
id_comment - 0
author_comment - 0
date_comment - 0
text_comment - 0
parent_comment - 3


In [10]:
# Просмотр данных
df_reviews = pd.DataFrame(data=reviews_list)
df_reviews.head()

,id,author_name,book_slug,title_book,review_url,author_book,date_creation,count_view,evaluation,title_review,text_review,count_like,count_comment
0,5071651,Чубаркина Наталья Михайловна,1008751306-na-izyaschnom-mify-v-iskusstve-sovr...,#На изящном: мифы в искусстве. Современный взг...,https://www.livelib.ru/review/5071651-na-izyas...,Мария Аборонова,11 мая 2025 г. 10:23,0,1,О мифах языком камеди- клаба.,1 из 5⭐О необходимости книги мы судим сначала ...,1,0
1,5071642,nat_phil,1000460354-atlant-raspravil-plechi-v-3-chastya...,Атлант расправил плечи. В 3 частях. Часть 2. И...,https://www.livelib.ru/review/5071642-atlant-r...,Айн Рэнд,11 мая 2025 г. 10:19,0,5,Гнев да и только,Эта книжка меня жутко бесила: в смысле автор п...,0,0
2,5071639,Крупкина Мария,1008875347-na-zapadnom-fronte-bez-peremen-erih...,На Западном фронте без перемен,https://www.livelib.ru/review/5071639-na-zapad...,Эрих Мария Ремарк,11 мая 2025 г. 10:18,0,5,СпойлерУжасы войны,Сразу после школы в 19 лет Пауля Боймера и его...,1,0
3,5071636,Жукова Елена,1010717885-sirena-morskih-glubin-sumi-han,Сирена морских глубин,https://www.livelib.ru/review/5071636-sirena-m...,Суми Хан,11 мая 2025 г. 10:17,0,3,"Эх, разочарование...","«Жить — значит выживать и выполнять свой долг,...",1,0
4,5071624,Booksevil,1005765304-lyubimets-kir-bulychjov,Любимец,https://www.livelib.ru/review/5071624-lyubimet...,Кир Булычёв,11 мая 2025 г. 10:11,0,3,"СпойлерСказ о том, как межпланетные лягухи нар...","Планета Земля. Она молода, красива и зелена. О...",2,0


In [ ]:
# Просмотр данных
df_comments_reviews = pd.DataFrame(data=comments_reviews_list)
df_comments_reviews.head()

,id_review,id_comment,author_comment,date_comment,text_comment,parent_comment


In [ ]:
# Сохранение данных
file_reviews = 'reviews.cvs'
df_reviews.to_csv(file_reviews)

file_comments_reviews = 'comments_reviews.cvs'
df_comments_reviews.to_csv(file_comments_reviews)

### Парсинг подборок.

In [3]:
# Структуры для хранения данных
collections_list = {
    'id': [],
    'author_name': [],
    'date_creation': [],
    'count_view': [],
    'count_book': [],
    'title': [],
    'collection_url': [],
    'desc': [],
    'count_like': [],
    'count_comment': [],
    'count_saved': [],
    'books': {
        'id_book': [],
        'title_book': [],
        'author_book': []
    }
}

comments_collections_list = {
    'id_collection': [],
    'id_comment': [],
    'author_comment': [],
    'date_comment': [],
    'text_comment': [],
    'parent_comment': []
}

In [4]:
# Настройки
headers = {'User-Agent': UserAgent().random}

In [5]:
# Парсинг описания подборки
def parse_collection_desc(desc_block):
    if not desc_block:
        return ''
    
    desc_div = desc_block.find('div', class_='description')
    if not desc_div:
        direct_text = desc_block.find('p')
        if direct_text and direct_text.text.strip() != "Нет описания":
            return direct_text.text.strip()
        return ''
    
    processed_text = []

    for element in desc_div.children:
        if element.name == 'p':
            # Обработка содержимое параграфа
            para_content = []
            for content in element.contents:
                if content.name in ['b', 'i', 'u', 'a', 'span']:
                    # Сохранение тегов форматирования
                    tag = content.name
                    text = content.get_text()
                    if tag == 'a' or tag == 'span':
                        para_content.append(text)
                    else:
                        para_content.append(f'<{tag}>{text}</{tag}>')
                elif content.name is None:
                    para_content.append(str(content))
            
            # Сбор текст параграфа и добавляем <br>
            para_text = ''.join(para_content).strip()
            if para_text:
                processed_text.append(para_text)
                processed_text.append('<br><br>')  # Двойной <br> для абзаца
        
        elif element.name == 'br':
            processed_text.append('<br>')
    
    # Объединение все части
    final_text = ''.join(processed_text)
    
    # Очистка лишние переносы
    final_text = re.sub(r'(<br>\s*){3,}', '<br><br>', final_text)
    final_text = final_text.strip()
    
    return final_text

In [6]:
# Парсинг информации о книге в подборке
def parse_book(book_div):
    if not book_div:
        return None
    
    # ID
    book_id = None
    cover_link = book_div.find('a', href=lambda x: x and ('/book/' in x or '/work/' in x))
    if cover_link:
        match = re.search(r'/(?:book|work)/(\d+)', cover_link['href'])
        book_id = match.group(1) if match else None

    # Название
    title = None
    title_element = book_div.find('a', class_='brow-book-name')
    if title_element:
        title = title_element.get('title', '').split(' - ')[-1].strip() or title_element.text.strip()

    # Авторы
    authors = []
    author_elements = book_div.find_all('a', class_='brow-book-author')
    for elem in author_elements:
        author_name = elem.get('title', '').strip() or elem.text.strip()
        if author_name and author_name not in authors:
            authors.append(author_name)

    if not authors and title_element and ' - ' in title_element.get('title', ''):
        possible_author = title_element['title'].split(' - ')[0].strip()
        if possible_author:
            authors.append(possible_author)

    return {
        'id_book': book_id,
        'title_book': title,
        'author_book': authors
    }

In [7]:
# Парсинг книг в подборке
def parse_collection_books(collection_url, collection_id):
    base_url = f'https://www.livelib.ru{collection_url}/listview/biglist'
    page = 1
    books = []

    while True:
        url = f'{base_url}/~{page}#books' if page > 1 else f'{base_url}#books'
        print(f'[{datetime.now().strftime("%H:%M:%S")}] Загрузка страницы {page}: {url} .')

        try:
            # Задержка перед загрузкой страницы с книгами
            time.sleep(get_medium_delay())

            response = requests.get(url, headers=headers, timeout=30)
            if response.status_code != 200:
                print(f'Ошибка загрузке страницы {page}: {url} .')
                time.sleep(get_long_delay())
                break

            soup = bs(response.text, 'html.parser')
            print(f'soup books, \n{soup}')

            # Проверка на блокировку
            if 'идёт слишком много запросов' in response.text:
                delay = get_extended_delay()
                print(f'Блокировка. Большая пауза ({delay:.1f} сек).')
                time.sleep(delay)
                continue

            # book_divs = soup.select('div[id^="my-selection-book-list-tr-"]')
            book_container = soup.find('div', class_='blist-biglist')
            book_divs = book_container.find_all('div', id=lambda x: x and x.startswith('my-selection-book-list-tr-'))
            print(f'Найдено {len(book_divs)} книг на странице {page}.')
            
            # Обработка книг с паузами
            for i, book_div in enumerate(book_divs, 1):
                if i % 5 == 0:
                    time.sleep(get_short_delay())

                book_info = parse_book(book_div)
                if book_info['id_book']:
                    books.append(book_info)

            # Проверка пагинации
            pagination = soup.find('div', id='booklist-pagination')
            if not pagination:
                break

            next_page = pagination.find('a', id=lambda x: x and 'a-list-page-next' in x)
            if not next_page or 'disabled' in next_page.get('class', []):
                break

            page += 1
        
        except Exception as e:
            print(f'Ошибка при парсинге страницы {page}: {str(e)} .')
            time.sleep(get_long_delay())
            break

    return books

In [8]:
# Парсинг комментариев к подборке
def parse_collection_comments(collection_id, collection_url):
    base_url = f'https://www.livelib.ru{collection_url}/comments'
    page = 1
    all_comments = []

    while True:
        url = f'{base_url}/~{page}#comments' if page > 1 else f'{base_url}#comments'
        print(f'[{datetime.now().strftime("%H:%M:%S")}] Парсинг комментариев страницы {page}: {url} .')

        try:
            # Задержка перед загрузкой комментариев
            time.sleep(get_medium_delay())

            response = requests.get(url, headers=headers, timeout=30)
            soup = bs(response.text, 'html.parser')

            # Проверка на блокировку
            if 'идёт слишком много запросов' in response.text:
                print('Блокировка. Большая пауза...')
                time.sleep(get_extended_delay())

            comments_block = soup.find('div', class_='commentnodes')
            if not comments_block:
                break

            comments = comments_block.find_all('div', class_=lambda x: x and 'comment-row' in x)
            print(f'Найдено {len(comments)} комментариев на странице.')

            # Обработка комментариев
            for i, comment in enumerate(comments, 1):
                if i % 3 == 0:
                    time.sleep(get_short_delay())

                try:
                    comment_id = comment.get('id', '').replace('comment', '')
                    comment_level = int(comment['class'][1].split('comment-level')[1])

                    author_tag = comment.find('a', class_='comment-user-avatar')
                    author = author_tag.get('title', 'Аноним').split('(')[0].strip() if author_tag else 'Аноним'

                    date_span = comment.find('span', class_='event-user-date')
                    date = date_span.get_text(strip=True).replace('&nbsp;', ' ') if date_span else None

                    text = ''
                    text_div = comment.find('div', classs_='commenttext')
                    if text_div:
                        for elem in text_div.find_all(['img', 'a', 'script']):
                            elem.decompose()
                        text = ' '.join(text_div.stripped_strings)
                    else:
                        text = None

                    parent = None
                    if comment_level > 0:
                        parent_block = comment.find_parent('div', id=lambda x: x and 'commentbranch' in x)
                        if parent_block:
                            parent = parent_block['id'].replace('commentbranch', '')

                    all_comments.append({
                        'id_collection': collection_id,
                        'id_comment': comment_id,
                        'author_comment': author,
                        'date_comment': date,
                        'text_comment': text,
                        'parent_comment': parent
                    })
                
                except Exception as e:
                    print(f'Ошибка парсинга комментария {comment.get('id')}: {str(e)} .')
                    continue

            # Проверка пагинации
            next_page = soup.find('a', id=lambda x: x and 'a-list-page-next' in x)
            if not next_page or 'disabled' in next_page.get('class', []):
                break

            page += 1

        except Exception as e:
            print(f'Ошибка при загрузке страницы {page}: {str(e)} .')
            time.sleep(get_long_delay())
            break

    return all_comments

In [9]:
# Основная функция парсинга подборок
def scrape_collections(start_page=1, max_pages=5):
    pagenum = start_page
    request_count = 0

    for _ in range(max_pages):
        try:
            url = f'https://www.livelib.ru/selections/~{pagenum}'
            print(f'\n[{datetime.now().strftime("%H:%M:%S")}] Страница {pagenum}: {url} .')

            # Задержки
            request_count += 1
            if request_count % 3 == 0:
                delay = get_long_delay()
                print(f'Большая пауза на {delay:.1f} сек...')
                time.sleep(delay)
            else:
                time.sleep(get_medium_delay())

            # Загрузка страницы с подборками
            page = requests.get(url, headers=headers, timeout=30)
            print(page.text)

            # Проверка на блокировку
            if 'идёт слишком много запросов' in page.text:
                delay = get_extended_delay()
                print(f'Блокировка. Большая пауза ({delay:.1f} сек).')
                time.sleep(delay)
                continue

            if page.status_code != 200:
                print(f'Ошибка загрузки страницы: {page.status_code}.')
                time.sleep(get_extended_delay())
                continue

            soup = bs(page.text, 'html.parser')
            print(soup)

            # Поиск подборок
            collection_cards = soup.find_all('article', class_=lambda x: x and 'bc-card' in x)
            if not collection_cards:
                print('Подборки не найдены, возможно блокировка или последняя страница.')
                time.sleep(get_extended_delay())
                break

            print(f'Найдено {len(collection_cards)} подборок.')

            # Обработка каждой подборки
            for i, collection_card in enumerate(collection_cards, 1):
                try:
                    # Задержка между подборками
                    if i % 2 == 0:
                        time.sleep(get_short_delay())

                    # ID
                    collection_id = collection_card.get('data-object_id')
                    if not collection_card:
                        continue

                    print(f'\nОбработка подборки {i}/{len(collection_cards)} (ID: {collection_id}).')

                    # Заголовок и URL
                    title_link = collection_card.find('a', class_='bc-card__text-block-title-link')
                    if not title_link:
                        continue

                    title = title_link.text.strip()
                    href = title_link['href']
                    full_url = urljoin('https://www.livelib.ru', href)

                    # Автор и дата
                    header_wrapper = collection_card.find('div', class_='bc-card__header-wrapper')
                    author_name = 'Неизвестно'
                    date_creation = 'Дата не указана'

                    if header_wrapper:
                        author_tag = header_wrapper.find('a', class_='bc-card__user-name-link')
                        author_name = author_tag.text.strip() if author_tag else author_name

                        date_tag = header_wrapper.find('time', class_=lambda x: x and 'bc-card__meta_type_time' in x)
                        date_creation = date_tag.text.strip() if date_tag else date_creation

                    # Количество просмотров
                    count_view = 0
                    count_view_item = collection_card.find('p', class_='bc-card__meta_type_views')
                    if count_view_item:
                        count_text = count_view_item.text.strip()
                        count_view = int(re.sub(r'[^\d]', '', count_text)) if count_text else 0

                    # Количество книг
                    count_book = 0
                    content_block = collection_card.find('div', class_='bc-card__text-block')
                    if content_block:
                        count_book_item = content_block.find('span', class_=lambda x: x and 'bc-card__meta_type_book-count' in x)
                        if count_book_item:
                            count_text = count_book_item.text.strip()
                            count_book = int(re.sub(r'[^\d]', '', count_text)) if count_text else 0

                    # Загрузка страницы подборки
                    time.sleep(get_medium_delay())
                    collection_page = requests.get(full_url, headers=headers, timeout=30)
                    if collection_page.status_code != 200:
                        print(f'Ошибка загрузки подборки: {collection_page.status_code}.')
                        continue

                    collection_soup = bs(collection_page.text, 'html.parser')

                    # Описание подборки
                    desc_block = collection_soup.find('div', class_='with-pad')
                    desc = parse_collection_desc(desc_block) if desc_block else ''

                    # Лайки, комментарии, сохранения
                    likes = comments = saves = 0
                    footer_container = collection_soup.find('div', class_='acb-selection-7761-main')

                    if footer_container:
                        # Лайки
                        like_span = footer_container.find('span', id='vote-plus-span-selection-7761')
                        if like_span:
                            like_text = like_span.get_text(strip=True)
                            likes = int(re.sub(r'[^\d]', '', like_text)) if like_text else 0

                        # Комментарии
                        comment_span = footer_container.find('span', class_='count')
                        if comment_span:
                            comment_text = comment_span.get_text(strip=True)
                            comments = int(re.sub(r'[^\d]', '', comment_text)) if comment_text else 0

                        # Сохранения
                        save_span = footer_container.find('span', class_='count-in-fav')
                        if save_span:
                            save_text = save_span.get_text(strip=True)
                            saves = int(re.sub(r'[^\d]', '', save_text)) if save_text else 0

                    # Парсинг книг в подборке
                    books = []
                    if count_book > 0:
                        print(f'Парсинг {count_book} книг в подборке.')
                        books = parse_collection_books(href, collection_id)
                        print(f'Успешно собрано {len(books)} книг.')

                    # Парсинг комментариев
                    collection_comments = []
                    if comments > 0:
                        print(f'Паринг {comments} комментариев.')
                        collection_comments = parse_collection_comments(collection_id, href)
                        print(f'Успешно собрано {len(collection_comments)} комментариев.')

                    # Сохранение данных
                    collections_list['id'].append(collection_id)
                    collections_list['author_name'].append(author_name)
                    collections_list['date_creation'].append(date_creation)
                    collections_list['count_view'].append(count_view)
                    collections_list['count_book'].append(count_book)
                    collections_list['title'].append(title)
                    collections_list['collection_url'].append(full_url)
                    collections_list['desc'].append(desc)
                    collections_list['count_like'].append(likes)
                    collections_list['count_comment'].append(comments)
                    collections_list['count_saved'].append(saves)

                    # Сохранение книг
                    for book in books:
                        collections_list['books']['id_book'].append(book['id_book'])
                        collections_list['books']['title_book'].append(book['title_book'])
                        collections_list['books']['author_book'].append(book['author_book'])

                    # Сохранение комментариев
                    for comment in collection_comments:
                        for key in comments_collections_list:
                            if key in comment:
                                comments_collections_list[key].append(comment[key])

                    print(f'Подборка {title} успешо обработана.')

                except Exception as e:
                    print(f'Ошибка обработки подборки: {str(e)} .')
                    time.sleep(get_long_delay())
                    continue

            pagenum += 1
        
        except Exception as e:
            print(f'Критическая ошибка: {str(e)} .')
            time.sleep(get_extended_delay())
            continue

In [ ]:
print(f'Начало парсинга: {datetime.now().strftime('%H:%M:%S')}')
scrape_collections(start_page=1, max_pages=5)
print(f'Завершение парсинга: {datetime.now().strftime('%H:%M:%S')}')

Начало парсинга: 21:32:40

[21:32:40] Страница 1: https://www.livelib.ru/selections/~1 .
<html>
<head>
<title>LiveLib</title>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<link rel="stylesheet" type="text/css" href="https://s.livelib.ru/css/style.1.css" />
<link rel="stylesheet" type="text/css" href="https://s.livelib.ru/skins/ll2015b/css/style.1.css" />
<link rel="stylesheet" type="text/css" href="https://s.livelib.ru/skins/ll2015b/css/stylev4.1.css" />

</head>
<body>

<div style="display: none;">
    <!--LiveInternet counter--><script type="text/javascript"><!--
        document.write("<img src='https://counter.yadro.ru/hit?r" +
                escape(document.referrer) + ((typeof (screen) == "undefined") ? "" :
                ";s" + screen.width + "*" + screen.height + "*" + (screen.colorDepth ?
                        screen.colorDepth : screen.pixelDepth)) + ";u" + escape(document.URL) +
                ";" + Math.random() +
                "' width=1 he

In [ ]:
# Просмотр количества пропусков
print('Количество нулевых значений в collections_list:')
for i in collections_list:
    print( i + " - " + str(collections_list[i].count(None)))

print()

print('Количество нулевых значений в comments_collections_list:')
for i in comments_collections_list:
    print( i + " - " + str(comments_collections_list[i].count(None)))

In [ ]:
# Просмотр данных
df_collections = pd.DataFrame(data=collections_list)
df_collections.head()

In [ ]:
# Просмотр данных
df_comments_collections = pd.DataFrame(data=comments_collections_list)
df_comments_collections.head()

In [ ]:
# Сохранение данных
file_collections = 'collections.cvs'
df_collections.to_csv(file_collections)

file_comments_collections = 'comments_collections.cvs'
df_comments_collections.to_csv(file_comments_collections)